<a href="https://colab.research.google.com/github/outlier/CNN_projects/blob/development/cats_dogs_mono.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

!pip install -q kaggle
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [2]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c dogs-vs-cats -p /content

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 33.1MB/s]
 97% 262M/271M [00:01<00:00, 171MB/s]
100% 271M/271M [00:02<00:00, 131MB/s]
 96% 524M/543M [00:02<00:00, 227MB/s]
100% 543M/543M [00:02<00:00, 203MB/s]


In [4]:
import zipfile
with zipfile.ZipFile("train.zip","r") as zip_ref:
    zip_ref.extractall("/content")
with zipfile.ZipFile("test1.zip","r") as zip_ref:
    zip_ref.extractall("/content")

In [5]:
import os
import numpy as np 
import matplotlib.pyplot as plt
import cv2

training_data = []

In [6]:
def prepare_training_data():
    CATEGORIES = ['cat','dog']
    for img in os.listdir('/content/train'):
      try:
        if 'cat' in img:
          class_num = CATEGORIES.index('cat')
        elif 'dog' in img:
          class_num = CATEGORIES.index('dog')
        image_array = cv2.imread(os.path.join('/content/train',img),cv2.IMREAD_GRAYSCALE)
        resized_array = cv2.resize(image_array,(50,50))
        training_data.append([resized_array,class_num])
      except Exception as e:
        pass
      
prepare_training_data()

In [7]:
print(len(training_data))
training_data[1][1]

25000


1

In [8]:
import random
random.shuffle(training_data)
X = []
Y = []

In [9]:
training_data[0]
training_data[1]

[array([[131, 137, 137, ...,  12,  12,   9],
        [138, 133, 139, ...,  62,  68,  67],
        [138, 138, 136, ...,  71,  71,  65],
        ...,
        [ 62,  60,  67, ...,  25,  23,  17],
        [ 56,  48,  51, ...,  38,  16,  15],
        [ 50,  43,  56, ...,  23,  20,  22]], dtype=uint8), 0]

In [10]:
for features,label in training_data:
  X.append(features)
  Y.append(label)

In [11]:
X = np.array(X).reshape(-1, 50,50,1)
Y = np.array(Y)
print(len(Y))

25000


In [12]:
import pickle 
feature_out = open('dog_cats_features.pickle','wb')
pickle.dump(X,feature_out)
feature_out.close()
label_out = open('dog_cats_label.pickle','wb')
pickle.dump(Y,label_out)
feature_out.close()

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Activation

# Normalize data 
X = X/255.0
X = tf.cast(X, tf.float32)

In [14]:
from tensorflow.keras.callbacks import TensorBoard
import time

MODEL_NAME = 'cats_dogs_mono_CNN-{}'.format(int(time.time()))
tensorboard = TensorBoard(log_dir="/content/logs/{}".format(MODEL_NAME))

In [15]:
model = Sequential()

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss="binary_crossentropy",optimizer="adam", metrics=["accuracy"])

model.fit(X,Y, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])

Epoch 1/10
704/704 [==============================] - 11s 6ms/step - loss: 0.6613 - accuracy: 0.5824 - val_loss: 0.5790 - val_accuracy: 0.7004
Epoch 2/10
704/704 [==============================] - 4s 5ms/step - loss: 0.5398 - accuracy: 0.7304 - val_loss: 0.4924 - val_accuracy: 0.7692
Epoch 3/10
704/704 [==============================] - 4s 5ms/step - loss: 0.4765 - accuracy: 0.7731 - val_loss: 0.4783 - val_accuracy: 0.7780
Epoch 4/10
704/704 [==============================] - 4s 5ms/step - loss: 0.4481 - accuracy: 0.7901 - val_loss: 0.4615 - val_accuracy: 0.7764
Epoch 5/10
704/704 [==============================] - 4s 5ms/step - loss: 0.4181 - accuracy: 0.8077 - val_loss: 0.4669 - val_accuracy: 0.7868
Epoch 6/10
704/704 [==============================] - 4s 5ms/step - loss: 0.3782 - accuracy: 0.8306 - val_loss: 0.4485 - val_accuracy: 0.7984
Epoch 7/10
704/704 [==============================] - 4s 5ms/step - loss: 0.3437 - accuracy: 0.8481 - val_loss: 0.4569 - val_accuracy: 0.7940
Epoch

In [16]:
from tensorflow import keras
#keras.utils.plot_model(model, "cat_dog.png")
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        36928     
_________________________________________________________________
activation_1 (Activation)    (None, 22, 22, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 7744)              0

In [23]:
def prepare_prediction_data(filename):
    try:
      image_array = cv2.imread(filename,cv2.IMREAD_GRAYSCALE)
      resized_array = cv2.resize(image_array, (50,50))
      #print(resized_array)
      resized_array = np.array(resized_array).reshape(-1, 50,50,1)
      return resized_array
    except Exception as e:
      print(e)
      
input_data = prepare_prediction_data('cat.jpg')
#print(input_data)      
prediction = model.predict([input_data])
print(prediction)

[[0.]]


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [26]:
model.save("{}.h5".format(MODEL_NAME))